In [2]:
using StatsBase, Random, LinearAlgebra, DecisionTree, DataFrames, MLDataUtils, ScikitLearn, CSV, Plots, Gurobi, JuMP

In [106]:
s = vec(Matrix(CSV.read("s.csv",DataFrame; header=0)));
s = broadcast(x -> Integer(x), s)
e = vec(Matrix(CSV.read("e.csv",DataFrame; header=0)));
e = broadcast(x -> Integer(x), e)
p = vec(Matrix(CSV.read("p.csv",DataFrame; header=0)));
t = vec(Matrix(CSV.read("t.csv",DataFrame; header=0)));
t = broadcast(x -> Integer(x), t)
y_neighbors = (Matrix(CSV.read("y_neighbors.csv",DataFrame; header=0)));
room_capacities = 80
Q = ones(5).*room_capacities #vec([5,5,5,5,5,5]);
M = 10^5;
f = 50;
q = 500;

In [18]:
X_new = (Matrix(CSV.read("X_new.csv",DataFrame; header=0)));
y_new = vec(Matrix(CSV.read("y_new.csv",DataFrame; header=0)));
y_new = broadcast(x -> Integer(x), y_new);
y_hat =  scenario_generation(y_neighbors);

# Scenario Generation

In [19]:
function scenario_generation(y_neighbors,method="knn")
    """
    generate scenarios of cancelation/non-cancelation based on historic data.
    Procedure will depend on whether the prescriptive method is knn or OCT
    
    @param y_neighbors: n x k matrix holding possible cancelation scenarios for each observation
    """
    n,k = size(y_neighbors)
    y_hat = zeros(n) 
    if method == "knn"
        for i=1:n
            p = sum(y_neighbors[i,:])/k
            y_hat[i] = p  #(rand() < p) ? 1 : 0 #1 with probability p, otherwise 0
        end
    end
    return y_hat
end

scenario_generation (generic function with 2 methods)

# Baseline Approach

The baseline approach will never allow a scenario where overbooking could happen.

In [71]:
function prescribe_bookings_baseline(Q,q,f,p,s,e,t)
    """
    A function that receives n booking requests over a time period 1,...,T and returns
    a decision on each booking (accept or reject) so as to maximize revenue.
    
    @param Q: vector length J. J is the number of room types we have (1 person, 2 person, etc.) Q_j is 
    the number of rooms available we have of type j 
    @param q: constant. this is the cost incurred if a customer is overbooked 
    @param f: constant. Money received if a booking is canceled 
    @param p: vector length n. The price of each booking 
    @param s: vector length n. The starting day of each booking
    @param e: vector length n. The ending day of each booking
    @param t: vector length n. This gives the number of people that the ith booking is for
    @param y_neighbors: binary matrix n X K. This is the historical neighbour data,
                        whether the person canceled or not
    @param M: big M constraints
    @param y_hat: estimate of cancelation probabilities 
    """
    J = length(Q)
    n = length(s)
    
    gurobi_env=Gurobi.Env(); #to suppress some Gurobi outputs
    model = Model(with_optimizer(Gurobi.Optimizer, gurobi_env))
    #set_optimizer_attribute(model, "OutputFlag", 0)
    #set_optimizer_attribute(model, "MIPGap", 0.1) #needed to ensure convergence

    
    @variable(model,z[i=1:n],Bin) #accept booking i or not

    #STRATEGY: Never allow num_people_booked > capacity 
    for i=1:n
        booked_people = 0 
        for i2=1:n
            if (s[i2] <= s[i]) && (e[i2] > s[i]) && (i2 !== i) && (t[i2] == t[i])
                booked_people += 1*z[i2]
            end
        end            
        @constraint(model,booked_people <= Q[t[i]])
    end
    
    @objective(model,Max, sum(z[i]*p[i] for i=1:n))
    optimize!(model)
    return objective_value(model) , value.(z)
end

prescribe_bookings_baseline (generic function with 1 method)

In [107]:
obj_baseline, z_baseline = prescribe_bookings_baseline(Q,q,f,p,s,e,t);

Academic license - for non-commercial use only - expires 2022-08-18
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2930 rows, 2930 columns and 757296 nonzeros
Model fingerprint: 0x896f080b
Variable types: 0 continuous, 2930 integer (2930 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 8e+01]
Found heuristic solution: objective 754997.12000
Presolve removed 1117 rows and 1082 columns
Presolve time: 1.39s
Presolved: 1813 rows, 1848 columns, 666847 nonzeros
Found heuristic solution: objective 836963.25000
Variable types: 0 continuous, 1848 integer (1845 binary)

Root relaxation: objective 9.484904e+05, 78 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

In [65]:
length(z_baseline)

2930

# kNN-based prescriptions

In [133]:
function prescribe_bookings_knn_new(Q,q,f,p,s,e,t,y_neighbors,y_hat)
    """
    A function that receives n booking requests over a time period 1,...,T and returns
    a decision on each booking (accept or reject) so as to maximize revenue.
    
    @param Q: vector length J. J is the number of room types we have (1 person, 2 person, etc.) Q_j is 
    the number of rooms available we have of type j 
    @param q: constant. this is the cost incurred if a customer is overbooked 
    @param f: constant. Money received if a booking is canceled 
    @param p: vector length n. The price of each booking 
    @param s: vector length n. The starting day of each booking
    @param e: vector length n. The ending day of each booking
    @param t: vector length n. This gives the number of people that the ith booking is for
    @param y_neighbors: binary matrix n X K. This is the historical neighbour data,
                        whether the person canceled or not
    @param M: big M constraints
    @param y_hat: estimate of whether people will cancel or not 
    """
    J = length(Q)
    n,K = size(y_neighbors)
    s = s .+ 1
    T = Int(maximum(s))
    
    gurobi_env=Gurobi.Env(); #to suppress some Gurobi outputs
    model = Model(with_optimizer(Gurobi.Optimizer, gurobi_env))
    #set_optimizer_attribute(model, "OutputFlag", 0)
    #set_optimizer_attribute(model, "MIPGap", 0.1) #needed to ensure convergence

    
    @variable(model,z[i=1:n],Bin) #accept booking i or not
    @variable(model,m[time=1:T,j=1:J]) #model ith min (for penalty terms)
    
    #STRATEGY: multiply expected surplus by q to penalise overbooking in objective
#     objective = 0
#     for i=1:n
#         expected_people = 0
#         for i2=1:n
#             if (s[i2] <= s[i]) && (e[i2] > s[i]) && (i2 !== i) && (t[i2] == t[i])
#                 expected_people += z[i2]*(1-y_hat[i2])*1
#             end
#         end   
#         @constraint(model, m[i] <= 0)
#         @constraint(model, m[i] <= Q[t[i]] - expected_people)
#         #objective_term = min(Q[t[i]] - expected_people,0)*(q/n)
#         objective += m[i]*(q+p[i]) #(q/n)
#     end
    objective = 0
    for time=1:T
        for j=1:J
            expected_people = 0
            for i=1:n
                if (s[i] <= time) && (e[i] > time) && (t[i] == j)
                    expected_people += z[i]*(1-y_hat[i])*1
                end
            end   
            @constraint(model, m[time,j] <= 0)
            @constraint(model, m[time,j] <= Q[j] - expected_people)
            #objective_term = min(Q[t[i]] - expected_people,0)*(q/n)
            if length(p[s.==time]) >= 1
                day_avg =  mean(p[s.==time])
            else
                day_avg = 0
            end
            objective += m[time,j]*(q+day_avg+f) #(q/n)
        end
    end
    
    objective += sum((1/K)*z[i]*(y_neighbors[i,k]*f + (1-y_neighbors[i,k])*p[i]) for k=1:K for i=1:n)
    
    @objective(model,Max, objective)
    optimize!(model)
    return objective_value(model) , value.(z), value.(m)
end

prescribe_bookings_knn_new (generic function with 2 methods)

In [101]:
mean(p[s.==22])

NaN

In [12]:
function prescribe_bookings_knn(Q,q,f,p,s,e,t,y_neighbors,y_hat)
    """
    A function that receives n booking requests over a time period 1,...,T and returns
    a decision on each booking (accept or reject) so as to maximize revenue.
    
    @param Q: vector length J. J is the number of room types we have (1 person, 2 person, etc.) Q_j is 
    the number of rooms available we have of type j 
    @param q: constant. this is the cost incurred if a customer is overbooked 
    @param f: constant. Money received if a booking is canceled 
    @param p: vector length n. The price of each booking 
    @param s: vector length n. The starting day of each booking
    @param e: vector length n. The ending day of each booking
    @param t: vector length n. This gives the number of people that the ith booking is for
    @param y_neighbors: binary matrix n X K. This is the historical neighbour data,
                        whether the person canceled or not
    @param M: big M constraints
    @param y_hat: estimate of cancelation probabilities 
    """
    J = length(Q)
    n,K = size(y_neighbors)
    
    gurobi_env=Gurobi.Env(); #to suppress some Gurobi outputs
    model = Model(with_optimizer(Gurobi.Optimizer, gurobi_env))
    #set_optimizer_attribute(model, "OutputFlag", 0)
    #set_optimizer_attribute(model, "MIPGap", 0.1) #needed to ensure convergence

    
    @variable(model,z[i=1:n],Bin) #accept booking i or not
    #@variable(model,b[i=1:n],Bin) #person i is overbooked or not
    #@variable(model,exp[i=1:n,j=1:J]>=0,Int) #number rooms occupied of type j when person i arrives
    
    #@constraint(model,[i=1:n,j=1:J],M*b[i]>=w[i,j]-Q[j]+1)

    #STRATEGY: Say that the expected number of guests cannot exceed capacity
    for i=1:n
        expected_people = 0 
        for i2=1:n
            if (s[i2] <= s[i]) && (e[i2] > s[i]) && (i2 !== i) && (t[i2] == t[i])
                expected_people += z[i2]*(1-y_hat[i2])*1
            end
        end            
        @constraint(model,expected_people <= Q[t[i]])
    end
    
    @objective(model,Max, sum(z[i]*(y_neighbors[i,k]*f + (1-y_neighbors[i,k])*p[i]) for k=1:K for i=1:n))
    optimize!(model)
    return objective_value(model) , value.(z)
end

prescribe_bookings_knn (generic function with 2 methods)

In [128]:
if length(s[s.==19]) == 0
    print("ewoier")
end

In [125]:
length(s[s.==11])

0

In [134]:
obj_knn, z_knn, m_knn = prescribe_bookings_knn_new(Q,q,f,p,s,e,t,y_neighbors,y_hat);

Academic license - for non-commercial use only - expires 2022-08-18
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 220 rows, 3040 columns and 7585 nonzeros
Model fingerprint: 0x25a46990
Variable types: 110 continuous, 2930 integer (2930 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e+01, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 8e+01]
Found heuristic solution: objective 752778.89000
Presolve removed 202 rows and 1879 columns
Presolve time: 0.01s
Presolved: 18 rows, 1161 columns, 3789 nonzeros
Found heuristic solution: objective 780265.98315
Variable types: 1 continuous, 1160 integer (987 binary)

Root relaxation: objective 8.555264e+05, 67 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

# OCT-based prescriptions

In [479]:
final_tree = IAI.read_json("final_tree");
#final_grid = IAI.read_json("final_grid");

In [480]:
function prescribe_bookings_oct(Q,q,f,p,s,e,t,y_neighbors,M)
    """
    A function that receives n booking requests over a time period 1,...,T and returns
    a decision on each booking (accept or reject) so as to maximize revenue.
    
    @param Q: vector length J. J is the number of room types we have (1 person, 2 person, etc.) Q_j is 
    the number of rooms available we have of type j 
    @param q: constant. this is the cost incurred if a customer is overbooked 
    @param f: constant. Money received if a booking is canceled 
    @param p: vector length n. The price of each booking 
    @param s: vector length n. The starting day of each booking
    @param e: vector length n. The ending day of each booking
    @param t: vector length n. This gives the number of people that the ith booking is for
    @param y_neighbors: binary matrix n X K. This is the historical neighbour data,
                        whether the person canceled or not
    @param M: big M constraints
    """
    J = length(Q)
    n,K = size(y_neighbors)
    
    gurobi_env=Gurobi.Env(); #to suppress some Gurobi outputs
    model = Model(with_optimizer(Gurobi.Optimizer, gurobi_env))
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    @variable(model,z[i=1:n],Bin) #accept booking i or not
    @variable(model,b[i=1:n],Bin) #person i is overbooked or not
    @variable(model,w[i=1:n,j=1:J]>=0) #number rooms occupied of type j when person i arrives
    
    @constraint(model,[i=1:n], z[i] <= 1-b[i])
    @constraint(model,[i=1:n,j=1:J],M*b[i]>=w[i,j]-Q[j]+1)

    @constraint(model,[i=1:n,j=1:J],w[i,j] ==  sum([z[l]*sum((1-y_neighbors[l,k]) for k=1:K) for l=1:n 
                                                    if ((s[l] <= s[i]) && (e[l] > s[i]) && (l!=i))]   )) 

    @objective(model,Max, sum(z[i]*(y_neighbors[i,k]*f + (1-y_neighbors[i,k])*p[i]) for k=1:K for i=1:n) -
                         q*sum(b[i] for i=1:n))
    optimize!(model)
    return objective_value(model) , value.(z)
end

prescribe_bookings_oct (generic function with 1 method)

# Evaluate Choice

In [14]:
function eval_choice(z,y,Q,q,f,p,s,e,t)
    """
    s/z has to be sorted by arrival data
    """
    J = length(Q)
    n,K = size(y_neighbors)
    total_profit = 0 
    num_overbooked = 0
    num_canceled = 0
    bookings_sold = 0
    for i=1:n
        if z[i] == 1
            if y[i] == 1
                total_profit += f
                num_canceled += 1
            else
                num_visitors = 0 
                for i2=1:(i-1)
                    if (e[i2] > s[i]) && (t[i]==t[i2]) 
                        num_visitors += z[i2]*(1-y[i2])
                    end
                end
                #println("num visitors: ",num_visitors)
                if num_visitors < Q[t[i]]
                    total_profit += p[i]
                    bookings_sold += 1
                else
                    num_overbooked += 1
                    total_profit -= q
                end
            end
        end
    end
    
    return total_profit,num_overbooked, num_canceled, bookings_sold
end

eval_choice (generic function with 1 method)

In [75]:
total_profit_baseline,num_overbooked_baseline, num_canceled_baseline, bookings_sold_baseline = eval_choice(z_baseline,y_new,Q,q,f,p,s,e,t)

(580868.6899999995, 0, 500, 863)

In [136]:
total_profit_knn,num_overbooked_knn, num_canceled_knn, bookings_sold_knn = eval_choice(z_knn,y_new,Q,q,f,p,s,e,t)

(286870.78000000014, 452, 988, 871)